In [ ]:
import pandas as pd
import re
import sys
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.tag import UnigramTagger
from nltk.corpus import brown
import webcolors
sys.path.append("..")
from ADA_JEX2017.Project.Functions.str_functions import *
from ADA_JEX2017.Project.Functions.pre_process import *


In [ ]:
path='../ADA_JEX2017/Project/Functions/'

In [ ]:
#our datafile exported to csv
data_file='./recipeInfo/recipeInfo_WestWhiteHorvitz_WWW2013.csv'
raw_data = pd.read_csv(data_file ,sep=';')

In [ ]:
with open(path+'units_list.txt', 'r') as f:
    units_list = [line.rstrip('\n') for line in f]
    
with open(path+'technique_list.txt', 'r') as f:
    techniques_list = [line.rstrip('\n') for line in f]

with open(path+'ingredient_list.txt', 'r') as f:
    ingredient_list = [line.rstrip('\n') for line in f]
    
lemmatizer = WordNetLemmatizer()
ingredient_list=[lemmatizer.lemmatize(token).lower() for token in ingredient_list]

In [ ]:
ingr_dataframe=raw_data[['title','ingredients_list','ingredients_bag-of-words']].copy().dropna()
ingr_dataframe = ingr_dataframe.reset_index(drop=True)
display(ingr_dataframe.head())
ingr_data_reduced=ingr_dataframe.head(100)
#we ignore upper case in the ingredients string
ingr_dataframe['ingredients_list']=ingr_dataframe['ingredients_list'].str.lower()

In [ ]:
fun_add_preprocess(ingr_dataframe,units_list)

In [ ]:
#-----test cell : test a specific receipe -------------
receipe=ingr_data_reduced.loc[0]['Recipe_preporcess']
print(receipe)
dic_ingr,dictec,wasted,wasted_numb=fun_extract_ingredients\
    (receipe,ingredient_list,techniques_list,units_list,to_gram=True)

In [ ]:
#----- calculate whole data frame ---------
ingr_data_reduced=ingr_dataframe.head(100)
all_dic=[]
not_used_ingr=[]
wastes=0
for index, row in ingr_dataframe.iterrows():
    recipe=row['Recipe_preporcess']
    dic_ingre,dictec,wasted_ingr,wasted_number=fun_extract_ingredients\
            (recipe,ingredient_list,techniques_list,units_list,to_gram=True)
    all_dic.append(dic_ingre)
    not_used_ingr.append(wasted_ingr)
    wastes=wastes+wasted_number

In [ ]:
# --- actually create the dataframe
ingredients_frame=pd.DataFrame(data=all_dic)
display(ingredients_frame.head(5))

In [ ]:
sum((ingredients_frame==0).sum(axis=1))

In [ ]:
print('There are : ',len(list(ingredients_frame)), 'ingredients')
ingred_used={}
for i in list(ingredients_frame):
    ingred_used[i]=sum(ingredients_frame[i].value_counts())

In [ ]:
occu=sorted((value,key) for (key,value) in ingred_used.items())
occu[::-1]